In [1]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

/Users/Timothy/repository/cs233_gtda_hw4/hw4_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device(type='cpu')

In [2]:
torch.cuda.is_available()
torch.cuda.device_count()
torch.cuda.get_device_name(0)

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
import torch
import numpy as np
import os
import os.path as osp
import tqdm
import matplotlib.pylab as plt
from torch import nn
from torch import optim
from collections import defaultdict

## Imports based on our ready-to-use code (after you pip-install the cs233_gtda_hw4 package)
from cs233_gtda_hw4.in_out.utils import make_data_loaders
from cs233_gtda_hw4.in_out.utils import save_state_dicts, load_state_dicts
from cs233_gtda_hw4.in_out import pointcloud_dataset
from cs233_gtda_hw4.in_out.plotting import plot_3d_point_cloud


## Imports you might use if you follow are scaffold code (it is OK to use your own stucture of the models)
from cs233_gtda_hw4.models import PointcloudAutoencoder
from cs233_gtda_hw4.models import PartAwarePointcloudAutoencoder
from cs233_gtda_hw4.models.point_net import PointNet
from cs233_gtda_hw4.models.mlp import MLP
from cs233_gtda_hw4.models.part_classifier import part_classifier

%load_ext autoreload
%autoreload 2

In [ ]:
##
## Fixed Settings (we do not expect you to change these)
## 

n_points = 1024  # number of points of each point-cloud
n_parts = 4      # max number of parts of each shape
n_train_epochs = 400

# Students: feel free to change below -ONLY- for the bonus Question:
# I.e., use THESE hyper-parameters when you train for the non-bonus questions.

part_lambda = 0.005  # for the part-aware AE you will be using (summing) two losses:
                     # chamfer + cross-entropy
                     # do it like this: chamfer + (part_lambda * cross-entropy), 
                     # i.e. we are scaling down the cross-entropy term
init_lr = 0.009  # initial learning-rate, tested by us with ADAM optimizer (see below)

In [ ]:
## Students: feel free to change below:

# batch-size of data loaders
batch_size = 128 # if you can keep this too as is keep it, 
                 # but if it is too big for your GPU, feel free to change it.

# which device to use: cpu or cuda?
#device = 'cpu'     # Note: only the "alternative" (slower) chamfer_loss in losses/nn_distance can run in cpu.
device = 'cuda'

top_in_dir = '../data/'
top_out_dir = '../data/out/'
if not osp.exists(top_out_dir):
    os.makedirs(top_out_dir)

In [ ]:
# PREPARE DATA:

loaders = make_data_loaders(top_in_dir, batch_size)

for split, loader in loaders.items():
    print('N-examples', split, len(loader.dataset))
    
# BUILD MODELS:
### TODO: Student on your own:
NUM_POINTS = 1024
NUM_CHANNELS = 3
LATENT_DIM = 128
# batch_size, num_channels, num_points

encoder = PointNet(NUM_CHANNELS)
decoder = MLP(LATENT_DIM, NUM_POINTS)
part_classifier = part_classifier(LATENT_DIM+NUM_CHANNELS, n_parts, 0.005)

## 1. Model

In [ ]:
part_aware_model = False # or True

if part_aware_model:
    xentropy = nn.CrossEntropyLoss()
    model = PartAwarePointcloudAutoencoder(encoder, decoder, part_classifier, part_lambda).to(device) # Students Work here
    model_tag = 'part_pc_ae'
else:
    model = PointcloudAutoencoder(encoder, decoder).to(device)  # Students Work here
    model_tag = 'pc_ae'

In [ ]:
print(model)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=init_lr)  # Students uncomment once you have defined your model

In [ ]:
## Train for multiple epochs your model.
# Students: the below for-loops are optional, feel free to structure your training 
# differently.

min_val_loss = np.Inf
out_file = osp.join(top_out_dir, model_tag + 'best_model.pth')
start_epoch = 1
train_loss = []
val_loss = []
test_loss = []

for epoch in tqdm.tqdm(range(start_epoch, start_epoch + n_train_epochs)):
    for phase in ['train', 'val', 'test']:        
        ### Students Work Here.
        recon_loss = model.train_for_one_epoch(loaders[phase], optimizer, device)
        print(phase, " loss: ", recon_loss)
        if phase == 'train':
            train_loss.append(recon_loss.item())
        elif phase == 'val':
            val_loss.append(recon_loss.item())
        elif phase == 'test':
            test_loss.append(recon_loss.item())
            
        if phase == 'val' and recon_loss < min_val_loss: # Save model if validation loss improved.
            min_val_loss = recon_loss
            save_state_dicts(out_file, epoch=epoch, model=model) # If you save the model like this, you can use the code below to load it. 

In [ ]:
fig, axes = plt.subplots(ncols=3, figsize=(20, 4))

axes[0].plot(range(n_train_epochs), train_loss, label='Train loss')
axes[0].set_title('Training Loss')
axes[1].plot(range(n_train_epochs), val_loss, label='Val loss')
axes[1].set_title('Validation Loss')
axes[2].plot(range(n_train_epochs), test_loss, label='Test loss')
axes[2].set_title('Test Loss')

In [ ]:
# Load model with best per-validation loss (uncomment when ready)
best_epoch = load_state_dicts(out_file, model=model)
print('per-validation optimal epoch', best_epoch)

In [ ]:
# Reconstruct from best model
recons, losses = model.reconstruct(loaders['test'], device=device)
recons = torch.cat(recons, dim=0)
losses = torch.cat(losses, dim=0)

In [ ]:
recon_losses = losses.cpu().numpy()
mean_loss = np.mean(recon_losses)
print('Reconstruction loss: ', mean_loss)

In [ ]:
# Students TODO: MAKE your plots and analysis

# 5 examples to visualize per questions (e, f)
examples_to_visualize = ['8a67fd47001e52414c350d7ea5fe2a3a',
                         '1e0580f443a9e6d2593ebeeedbff73b',
                         'd3562f992aa405b214b1fd95dbca05',
                         '4e8d8792a3a6390b36b0f2a1430e993a',
                         '58479a7b7c157865e68f66efebc71317']
# You can (also) use the function for the reconstructions or the part-predictions 
# (for the latter check the kwargs parameter 'c' of matplotlib.
# plot_3d_point_cloud, eg. try plot_3d_point_cloud(loaders['test'].dataset.pointclouds[0])

for example in examples_to_visualize:
    print("Example: ", example)
    index = np.where(loaders['test'].dataset.model_names == example)
    index = np.int(index[0])
    
    
    # Plot side by side
    #f, axarr = plt.subplots(1,2)
    
    orig = plot_3d_point_cloud(loaders['test'].dataset.pointclouds[index], title = 'original')
    recon = plot_3d_point_cloud(recons[index].cpu(), title = 'reconstructed')
    
    #axarr[1].imshow(recon)




#model.eval()   # Do not forget this.! We are not training any more (OK, since we do not 
               # have batch-norm, drop-out etc. this is not so important, however it is good standard 
               # practice)

In [ ]:
# Best and worst
min_example = recon_losses.argmin()
max_example = recon_losses.argmax()

print("Best Example: ", example)
print("Chamfer distance: ", recon_losses.min())
orig = plot_3d_point_cloud(loaders['test'].dataset.pointclouds[min_example], title = 'original')
recon = plot_3d_point_cloud(recons[min_example].cpu(), title = 'reconstructed')

print("Worst Example: ", example)
print("Chamfer distance: ", recon_losses.max())
orig = plot_3d_point_cloud(loaders['test'].dataset.pointclouds[max_example], title = 'original')
recon = plot_3d_point_cloud(recons[max_example].cpu(), title = 'reconstructed')

In [ ]:
model.eval()   # Do not forget this.! We are not training any more (OK, since we do not 
               # have batch-norm, drop-out etc. this is not so important, however it is good standard 
               # practice)

In [ ]:
# Last, save the latent codes of the test data and go to the 
# measuring_part_awareness and tsne_plot_with_latent_codes code.

# Students TODO: Extract the latent codes and save them, so you can analyze them later.
latent_codes, test_names = model.extract_latent_code(loaders['test'], device=device)   

latent_codes = latent_codes.cpu()

np.savez(osp.join(top_out_dir, model_tag +'_latent_codes'), 
         latent_codes=latent_codes, 
         test_names=test_names)

## 2. Part-aware model

In [ ]:
part_aware_model = True # or True

if part_aware_model:
    xentropy = nn.CrossEntropyLoss()
    model = PartAwarePointcloudAutoencoder(encoder, decoder, part_classifier, part_lambda).to(device) # Students Work here
    model_tag = 'part_pc_ae'
else:
    model = PointcloudAutoencoder(encoder, decoder).to(device)  # Students Work here
    model_tag = 'pc_ae'

In [ ]:
print(model)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=init_lr)  # Students uncomment once you have defined your model

In [ ]:
## Train for multiple epochs your model.
# Students: the below for-loops are optional, feel free to structure your training 
# differently.

min_val_loss = np.Inf
out_file = osp.join(top_out_dir, model_tag + 'best_model.pth')
start_epoch = 1

train_xe_loss = []
val_xe_loss = []
test_xe_loss = []


train_chamfer_loss = []
val_chamfer_loss = []
test_chamfer_loss = []

train_joint_loss = []
val_joint_loss = []
test_joint_loss = []


for epoch in tqdm.tqdm(range(start_epoch, start_epoch + n_train_epochs)):
    for phase in ['train', 'val', 'test']:        
        ### Students Work Here.
        joint_loss, chamfer_loss, xe_loss = model.train_for_one_epoch(loaders[phase], optimizer, device)
        print(phase, "joint loss: ", joint_loss, " chamfer loss: ", chamfer_loss, "xe loss: ", xe_loss)
        if phase == 'train':
            train_chamfer_loss.append(chamfer_loss.item())
            train_joint_loss.append(joint_loss.item())
            train_xe_loss.append(xe_loss.item())
        elif phase == 'val':
            val_chamfer_loss.append(chamfer_loss.item())
            val_joint_loss.append(joint_loss.item())
            val_xe_loss.append(xe_loss.item())
        elif phase == 'test':
            test_chamfer_loss.append(chamfer_loss.item())
            test_joint_loss.append(joint_loss.item())
            test_xe_loss.append(xe_loss.item())
            
        if phase == 'val' and joint_loss < min_val_loss: # Save model if validation loss improved.
            min_val_loss = joint_loss
            save_state_dicts(out_file, epoch=epoch, model=model) # If you save the model like this, you can use the code below to load it. 

In [ ]:
fig, axes = plt.subplots(ncols=3, figsize=(20, 4))

fig.suptitle("Chamfer loss")
axes[0].plot(range(n_train_epochs), train_chamfer_loss, label='Train loss')
axes[0].set_title('Training Loss')
axes[1].plot(range(n_train_epochs), val_chamfer_loss, label='Val loss')
axes[1].set_title('Validation Loss')
axes[2].plot(range(n_train_epochs), test_chamfer_loss, label='Test loss')
axes[2].set_title('Testing Loss')

In [ ]:
fig, axes = plt.subplots(ncols=3, figsize=(20, 4))

fig.suptitle("Cross Entropy loss")
axes[0].plot(range(n_train_epochs), train_xe_loss, label='Train loss')
axes[0].set_title('Training Loss')
axes[1].plot(range(n_train_epochs), val_xe_loss, label='Val loss')
axes[1].set_title('Validation Loss')
axes[2].plot(range(n_train_epochs), test_xe_loss, label='Test loss')
axes[2].set_title('Testing Loss')

In [ ]:
# Load model with best per-validation loss (uncomment when ready)
best_epoch = load_state_dicts(out_file, model=model)
print('per-validation optimal epoch', best_epoch)

In [ ]:
# Reconstruct from best model
recons, losses, recon_losses, pred_labels = model.reconstruct(loaders['test'], device=device)
recons = torch.cat(recons, dim=0)
recon_losses = torch.stack(recon_losses)
pred_labels = torch.cat(pred_labels, dim=0).cpu()

In [ ]:
_, pred_labels = torch.max(pred_labels, 1)
pred_labels = pred_labels.cpu().numpy()

In [ ]:
recon_losses = recon_losses.cpu().numpy()
mean_loss = np.mean(recon_losses)
print('Reconstruction loss: ', mean_loss)

In [ ]:
# Students TODO: MAKE your plots and analysis

# 5 examples to visualize per questions (e, f)
examples_to_visualize = ['8a67fd47001e52414c350d7ea5fe2a3a',
                         '1e0580f443a9e6d2593ebeeedbff73b',
                         'd3562f992aa405b214b1fd95dbca05',
                         '4e8d8792a3a6390b36b0f2a1430e993a',
                         '58479a7b7c157865e68f66efebc71317']
# You can (also) use the function for the reconstructions or the part-predictions 
# (for the latter check the kwargs parameter 'c' of matplotlib.
# plot_3d_point_cloud, eg. try plot_3d_point_cloud(loaders['test'].dataset.pointclouds[0])

for example in examples_to_visualize:
    print("Example: ", example)
    index = np.where(loaders['test'].dataset.model_names == example)
    index = np.int(index[0])
    
    orig = plot_3d_point_cloud(loaders['test'].dataset.pointclouds[index], title = 'original', c = loaders['test'].dataset.part_masks[index])
    recon = plot_3d_point_cloud(recons[index].cpu(), title = 'reconstructed', c = pred_labels[index])
    




#model.eval()   # Do not forget this.! We are not training any more (OK, since we do not 
               # have batch-norm, drop-out etc. this is not so important, however it is good standard 
               # practice)

In [ ]:
tolerance = 1e-10
accuracy = (np.abs(pred_labels - loaders['test'].dataset.part_masks) < tolerance).mean(axis=1)

In [ ]:
accuracy.mean()

In [ ]:
# Best and worst
min_example = accuracy.argmin()
max_example = accuracy.argmax()


print("Worst Example: ", example)
print("Accuracy: ", accuracy.min())

orig = plot_3d_point_cloud(loaders['test'].dataset.pointclouds[min_example], title = 'original',c = loaders['test'].dataset.part_masks[min_example])
recon = plot_3d_point_cloud(recons[min_example].cpu(), title = 'reconstructed', c = pred_labels[min_example])

print("Best Example: ", example)
print("Accuracy: ", accuracy.max())

orig = plot_3d_point_cloud(loaders['test'].dataset.pointclouds[max_example], title = 'original',c = loaders['test'].dataset.part_masks[max_example])
recon = plot_3d_point_cloud(recons[max_example].cpu(), title = 'reconstructed', c = pred_labels[max_example])